In [23]:
from datasets import load_dataset
import os
import tarfile
from pathlib import Path
import glob
from itertools import chain
import polars as pl
import numpy as np
!uv pip install bs4
import json

Using Python 3.13.11 environment at: /Users/sahila/Documents/Self Learning/transformers_hf/.venv
Audited 1 package in 15ms


In [2]:
BASE_DIR = Path(os.getcwd())

In [3]:
dataset = load_dataset("fka/awesome-chatgpt-prompts")
dataset

DatasetDict({
    train: Dataset({
        features: ['act', 'prompt', 'for_devs', 'type', 'contributor'],
        num_rows: 810
    })
})

In [4]:
dataset['train'][0]

{'act': 'Ethereum Developer',
 'prompt': 'Imagine you are an experienced Ethereum developer tasked with creating a smart contract for a blockchain messenger. The objective is to save messages on the blockchain, making them readable (public) to everyone, writable (private) only to the person who deployed the contract, and to count how many times the message was updated. Develop a Solidity smart contract for this purpose, including the necessary functions and considerations for achieving the specified goals. Please provide the code and any relevant explanations to ensure a clear understanding of the implementation.',
 'for_devs': True,
 'type': 'TEXT',
 'contributor': 'ameya-2003'}

In [5]:
dataset['train'].shuffle(seed= 37).select(range(100))

Dataset({
    features: ['act', 'prompt', 'for_devs', 'type', 'contributor'],
    num_rows: 100
})

In [6]:
## split into train, test
dataset['train'].train_test_split(
    train_size = 0.8, seed = 42
)

DatasetDict({
    train: Dataset({
        features: ['act', 'prompt', 'for_devs', 'type', 'contributor'],
        num_rows: 648
    })
    test: Dataset({
        features: ['act', 'prompt', 'for_devs', 'type', 'contributor'],
        num_rows: 162
    })
})

### Creating your own dataset using a sgm file

In [7]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/reuters21578-mld/reuters21578.tar.gz

--2025-12-30 12:30:44--  https://archive.ics.uci.edu/ml/machine-learning-databases/reuters21578-mld/reuters21578.tar.gz
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘reuters21578.tar.gz’

reuters21578.tar.gz     [  <=>               ]   7.77M  20.5MB/s    in 0.4s    

2025-12-30 12:30:44 (20.5 MB/s) - ‘reuters21578.tar.gz’ saved [8150596]



In [8]:
import tarfile
with tarfile.open(BASE_DIR/"my_model"/"datasets"/"reuters21578.tar.gz", 'r:gz') as tar:
    tar.extractall(path=BASE_DIR/"my_model"/"datasets"/"destination_folder")

/var/folders/pz/5t94392x1t1_xpdnb9_w0_0r0000gn/T/ipykernel_44885/4224340313.py:3: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=BASE_DIR/"my_model"/"datasets"/"destination_folder")


In [9]:
from bs4 import BeautifulSoup

def parse_sgm(file_path):
    with open(file_path, 'r', encoding='latin-1') as f:
        content = f.read()

    soup = BeautifulSoup(content, 'html.parser')

    documents = []
    for reuters in soup.find_all('reuters'):
        doc = {
            'title': reuters.title.get_text() if reuters.title else '',
            'body': reuters.body.get_text() if reuters.body else '',
            'topics': [topic.get_text() for topic in reuters.find_all('d')]
        }
        documents.append(doc)

    return documents

all_sgm_files = glob.glob(str(BASE_DIR/"my_model"/"datasets"/"destination_folder")+"/*.sgm")
articles = []
for each_file in all_sgm_files:
    articles.append(parse_sgm(each_file))
articles = list(chain.from_iterable(articles))

In [38]:
def save_to_json(data, filename):
    with open(filename, 'w') as f:
        for article in data:
            f.write(json.dumps(article) + "\n")

train_pct, val_pct = 0.8, 0.1
# train_articles = 
# train_articles = articles
train_articles = articles[0: int(train_pct* len(articles))]
val_articles =  articles[int(train_pct* len(articles)): int((train_pct + val_pct)*len(articles))]
test_articles =  articles[int((train_pct + val_pct)*len(articles)):]

len(train_articles) + len(val_articles) + len(test_articles)

save_to_json(train_articles, str(BASE_DIR/"my_model"/"datasets"/"processed_output"/"train.json"))
save_to_json(test_articles, str(BASE_DIR/"my_model"/"datasets"/"processed_output"/"test.json"))
save_to_json(val_articles, str(BASE_DIR/"my_model"/"datasets"/"processed_output"/"val.json"))

## load a parquet file

In [ ]:
np.random.seed(42)
idf = pl.DataFrame({'a': range(1_000_000),'b': [np.random.choice(['s','a','h','i','l']) for x in range(1_000_000)]})
idf.write_parquet(str(BASE_DIR/"my_model"/"datasets"/"custom_df.parquet"))

In [ ]:
my_df = load_dataset("parquet",data_files=str(BASE_DIR/"my_model"/"datasets"/"custom_df.parquet"))

In [ ]:
## The function to compute the square of a column. The output has to be formatted as a dict
def compute_square(row)->dict:
    return {'c': row['a']**2}

my_df = my_df.map(compute_square, num_proc=16)

In [ ]:
my_df['train']

In [ ]:
## dataset sharding for parallel processing
print(my_df.shape)

my_df['train'].shard(num_shards=6, index = 0)
